In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
import yfinance as yf
import time
#https://stackoverflow.com/questions/14463277/how-to-disable-python-warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
nvidia = yf.Ticker('NVDA')
test = nvidia.history( interval='5m')

In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1472 entries, 2020-12-28 09:30:00-05:00 to 2021-01-25 15:15:19-05:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          1472 non-null   float64
 1   High          1472 non-null   float64
 2   Low           1472 non-null   float64
 3   Close         1472 non-null   float64
 4   Volume        1472 non-null   int64  
 5   Dividends     1472 non-null   int64  
 6   Stock Splits  1472 non-null   int64  
dtypes: float64(4), int64(3)
memory usage: 92.0 KB


In [6]:
def price_mage(ticker):
    while int(time.asctime().split()[3].replace(':','')) < 163000:
        test =  yf.download(tickers= ticker,interval ='5m',period = '1mo')
        print(test.shape)
        #zip learned: https://stackoverflow.com/questions/1663807/how-to-iterate-through-two-lists-in-parallel
        obv = []
        prev = 0
        obv_num = 0 
        for close,volume in  zip(test['Close'], test['Volume']):
            if close> prev:
                obv_num += volume
                prev = close
                obv.append(obv_num)
            elif close < prev:
                obv_num -= volume
                prev = close 
                obv.append(obv_num)
            else:
                obv.append(obv_num)


        test['OBV'] = obv

        prev = 0
        Buy = []
        for close in test['Close']:
            if close > prev:
                Buy.append(1)
                prev = close 
            elif close <= prev:
                Buy.append(0)
                prev = close
        test['Buy'] = Buy

        test['Rolling_hour_average'] =test['Close'].rolling(12).mean()

        test['Above_average'] = [1 if close > average else 0 for close,average in zip(test['Close'],test['Rolling_hour_average'])]

        #Code and library adapted from https://blog.quantinsti.com/adx-indicator-python/
        high_low = []
        abs_high_prev = []
        abs_low_prev = []
        prev = 0
        print(test.shape)
        for close,high,low in zip(test['Close'],test['High'],test['Low']):
            high_low.append(abs(high-low))
            abs_high_prev.append(abs(high-prev))
            abs_low_prev.append(abs(low-prev))
            prev = close 
        from ta.trend import ADXIndicator
        print(test.shape)
        test['high_low'] = high_low
        test['abs_high_prev'] = abs_high_prev
        test['abs_low_prev'] = abs_low_prev
        print(test.shape)
        adx = ADXIndicator(test['high_low'],test['abs_high_prev'],test['abs_low_prev'])
        test['adx'] = adx.adx()
        test.drop(columns= ['high_low','abs_high_prev','abs_low_prev'], inplace = True)

        #https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html
        from ta.trend import AroonIndicator
        aroon = AroonIndicator(test['Close'])
        test['Aroon_Indicator'] = aroon.aroon_indicator()

        from ta.trend import MACD
        Macd = MACD(test['Close'])
        test['MACD'] = Macd.macd()

        test.dropna(inplace = True)


        features = ['Close','OBV','Rolling_hour_average','Above_average','adx','Aroon_Indicator','MACD']
        X = test[features]
        y = test['Buy'].shift(-1)
        X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.7, random_state = 6)
        model = LogisticRegression()
        model.fit(X_train,y_train)
        print(f' Model Score for {time.asctime()}: {model.score(X_test,y_test)}')
        time.sleep(30)

In [ ]:
test.tail()

In [ ]:
features = ['Close','OBV','Rolling_hour_average','Above_average','adx','Aroon_Indicator','MACD']
X = test[features]
y = test['Buy']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.7, random_state = 6)

In [8]:
model = LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.5222222222222223